In [1]:
import pandas as pd
import numpy as np
import math
import warnings
warnings.filterwarnings("ignore")

In [7]:
order_report = pd.read_excel("Company ShopX - Order Report.xlsx")
product_weight = pd.read_excel("Company ShopX - Product Weight.xlsx")
product_weight.drop_duplicates(inplace=True)
warehouse_customer = pd.read_excel("Company ShopX - Warehouse&Customer Pin Code and Zone details.xlsx")
cc_invoice = pd.read_excel("Courier Company - Invoice.xlsx")
cc_rates = pd.read_excel("Courier Company - Rates.xlsx").T


df = pd.merge(order_report,product_weight,how="left",on="Product Code")


PermissionError: [Errno 13] Permission denied: 'Company ShopX - Product Weight.xlsx'

In [ ]:
 

grouped = df.groupby("Order ID")[["Total Weight"]].sum()
merged = pd.merge(cc_invoice,grouped,how="left",on="Order ID")
merged["Total Weight"] = merged["Total Weight"]/1000
merged1 = pd.merge(merged,warehouse_customer,how = "left",on = "Customer Area Code")
Finalmergeddata = merged1.drop_duplicates()

In [4]:
order_report

,Order ID,Product Code,Units Ordered
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0
...,...,...,...
395,2001806229,8904223818942,1.0
396,2001806229,8904223818850,1.0
397,2001806226,8904223818850,2.0
398,2001806210,8904223816214,1.0


In [6]:
product_weight

,Product Code,Product Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120
...,...,...
61,8904223819505,210
62,8904223819499,210
63,8904223819512,210
64,8904223819543,300


In [21]:
r = math.ceil((Finalmergeddata["Total Weight"].max())/0.5)
ls = []
for i in range(1,r+1):
    ls.append(i*0.5)
    print(ls)

def weight_slab(value):
    for j in ls:
        if value <= j:
            return j
    return value  


Finalmergeddata["New Weight"] = Finalmergeddata["Total Weight"].apply(lambda x: weight_slab(x))
print(Finalmergeddata)

[0.5]
[0.5, 1.0]
[0.5, 1.0, 1.5]
[0.5, 1.0, 1.5, 2.0]
[0.5, 1.0, 1.5, 2.0, 2.5]
[0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5]
     AWB Code (Airway Bill Number)     Order ID  Chargeable Weight  \
0                     1091117222124  2001806232               1.30   
1                     1091117222194  2001806273               1.00   
2                     1091117222931  2001806408               2.50   
3                     1091117223244  2001806458               1.00   
4                     1091117229345  2001807012               0.15   
..                              ...         ...                ...   
167                   1091118551656  2001812941               0.73   
168                   1091117614452  2001809383               0.50   
169                   1091120922803  2001820978               0.50   
170                   1091121844806  2001811475               0.50   
172                   1091121846136  2001811305               0.50   

     Store H

In [22]:
def rates(row):
    base_rates = {
        ('a', 'Forward charges'): (29.5, 23.6),
        ('a', 'Forward and RTO charges'): (13.6, 23.6),
        ('b', 'Forward charges'): (33, 28.3),
        ('b', 'Forward and RTO charges'): (20.5, 28.3),
        ('c', 'Forward charges'): (40.1, 38.9),
        ('c', 'Forward and RTO charges'): (31.9, 38.9),
        ('d', 'Forward charges'): (45.4, 44.8),
        ('d', 'Forward and RTO charges'): (41.3, 44.8),
        ('e', 'Forward charges'): (56.6, 55.5),
        ('e', 'Forward and RTO charges'): (50.7, 55.5)
    }

    zone_type = (row["Delivery Zone_y"], row["Freight Type"])
    base_rate, additional_rate = base_rates.get(zone_type, (0, 0))

    if row["New Weight"] == 0.5:
        return base_rate
    else:
        return base_rate + (row["ab"] * additional_rate)

In [23]:
Finalmergeddata["ab"] = (Finalmergeddata["New Weight"]//0.5) - 1
def rates(row):
    if (row["Delivery Zone_y"] == "a") and (row["Freight Type"] == "Forward charges") and (row["New Weight"] == 0.5):
        return  29.5
    elif (row["Delivery Zone_y"] == "a") and (row["Freight Type"] == "Forward charges") and (row["New Weight"] > 0.5):
        return  29.5 + (row["ab"] * 23.6)
    elif (row["Delivery Zone_y"] == "a") and (row["Freight Type"] == "Forward and RTO charges") and (row["New Weight"] == 0.5):
        return  13.6
    elif (row["Delivery Zone_y"] == "a") and (row["Freight Type"] == "Forward and RTO charges") and (row["New Weight"] > 0.5):        
        return 13.6 + (row["ab"] * 23.6)
        
            
    elif (row["Delivery Zone_y"] == "b") and (row["Freight Type"] == "Forward charges") and (row["New Weight"] == 0.5):
        return 33
    elif (row["Delivery Zone_y"] == "b") and (row["Freight Type"] == "Forward charges") and (row["New Weight"] > 0.5):
        return  33 + (row["ab"] * 28.3)
    elif (row["Delivery Zone_y"] == "b") and (row["Freight Type"] == "Forward and RTO charges") and (row["New Weight"] == 0.5):
        return 20.5
    elif (row["Delivery Zone_y"] == "b") and (row["Freight Type"] == "Forward and RTO charges") and (row["New Weight"] > 0.5):
        return  20.5 + (row["ab"] * 28.3)
        
        
    elif (row["Delivery Zone_y"] == "c") and (row["Freight Type"] == "Forward charges") and (row["New Weight"] == 0.5):
        return 40.1
    elif (row["Delivery Zone_y"] == "c") and (row["Freight Type"] == "Forward charges") and (row["New Weight"] > 0.5):
        return  40.1 + (row["ab"] * 38.9)
    elif (row["Delivery Zone_y"] == "c") and (row["Freight Type"] == "Forward and RTO charges") and (row["New Weight"] == 0.5):
        return  31.9
    elif (row["Delivery Zone_y"] == "c") and (row["Freight Type"] == "Forward and RTO charges") and (row["New Weight"] > 0.5):
        return  31.9 + (row["ab"] * 38.9)
    
    
    elif (row["Delivery Zone_y"] == "d") and (row["Freight Type"] == "Forward charges") and (row["New Weight"] == 0.5):
        return  45.4
    elif (row["Delivery Zone_y"] == "d") and (row["Freight Type"] == "Forward charges") and (row["New Weight"] > 0.5):
        return  45.4 + (row["ab"] * 44.8)
    elif (row["Delivery Zone_y"] == "d") and (row["Freight Type"] == "Forward and RTO charges") and (row["New Weight"] == 0.5):
        return 41.3
    elif (row["Delivery Zone_y"] == "d") and (row["Freight Type"] == "Forward and RTO charges") and (row["New Weight"] > 0.5):
        return 41.3 + (row["ab"] * 44.8)
       
    
    elif (row["Delivery Zone_y"] == "e") and (row["Freight Type"] == "Forward charges") and (row["New Weight"] == 0.5):
        return  56.6
    elif (row["Delivery Zone_y"] == "e") and (row["Freight Type"] == "Forward charges") and (row["New Weight"] > 0.5):
        return (56.6 + (row["ab"] * 55.5))
    elif (row["Delivery Zone_y"] == "e") and (row["Freight Type"] == "Forward and RTO charges") and (row["New Weight"] == 0.5):
        return 50.7
    elif (row["Delivery Zone_y"] == "e") and (row["Freight Type"] == "Forward and RTO charges") and (row["New Weight"] > 0.5):
        return (50.7 + (row["ab"] * 55.5))
        
    else:
        return 0


Finalmergeddata["Our Amount5"] = Finalmergeddata.apply(rates, axis=1)
print(Finalmergeddata)

     AWB Code (Airway Bill Number)     Order ID  Chargeable Weight  \
0                     1091117222124  2001806232               1.30   
1                     1091117222194  2001806273               1.00   
2                     1091117222931  2001806408               2.50   
3                     1091117223244  2001806458               1.00   
4                     1091117229345  2001807012               0.15   
..                              ...         ...                ...   
167                   1091118551656  2001812941               0.73   
168                   1091117614452  2001809383               0.50   
169                   1091120922803  2001820978               0.50   
170                   1091121844806  2001811475               0.50   
172                   1091121846136  2001811305               0.50   

     Store House Pincode_x  Customer Area Code Delivery Zone_x  \
0                   121003              507101               d   
1                   121003 

In [24]:
Finalmergeddata["Net"] = round(Finalmergeddata["Our Amount5"] - Finalmergeddata["Total Amount (Rs.)"],2)
def status(row):
    if row["Net"] == 0:
        return "Correctly Charged"
    elif row["Net"] > 0:
        return "Under Charged"
    else:
        return "Over Charged"

Finalmergeddata["Status"] = Finalmergeddata.apply(status,axis= 1)
print(Finalmergeddata)
Finalmergeddata["Net"]

     AWB Code (Airway Bill Number)     Order ID  Chargeable Weight  \
0                     1091117222124  2001806232               1.30   
1                     1091117222194  2001806273               1.00   
2                     1091117222931  2001806408               2.50   
3                     1091117223244  2001806458               1.00   
4                     1091117229345  2001807012               0.15   
..                              ...         ...                ...   
167                   1091118551656  2001812941               0.73   
168                   1091117614452  2001809383               0.50   
169                   1091120922803  2001820978               0.50   
170                   1091121844806  2001811475               0.50   
172                   1091121846136  2001811305               0.50   

     Store House Pincode_x  Customer Area Code Delivery Zone_x  \
0                   121003              507101               d   
1                   121003 

0       0.0
1      -0.0
2       0.0
3       0.0
4       0.0
       ... 
167   -57.2
168   -37.9
169    15.9
170    79.1
172    15.9
Name: Net, Length: 124, dtype: float64

In [25]:
Finalmergeddata["AWB Number"] = Finalmergeddata['AWB Code (Airway Bill Number) '] 
Finalmergeddata["Total weight as per ShopX (KG)"] = Finalmergeddata["Total Weight"]
Finalmergeddata["Weight slab as per ShopX (KG)"] = Finalmergeddata["New Weight"]
Finalmergeddata["Total weight as per Courier Company (KG)"] = Finalmergeddata["Chargeable Weight"]
Finalmergeddata["Weight slab charged by Courier Company (KG)"] = Finalmergeddata["Chargeable Weight"].apply(lambda x: weight_slab(x))
Finalmergeddata["Delivery Zone as per ShopX"] = Finalmergeddata["Delivery Zone_y"]
Finalmergeddata["Delivery Zone charged by Courier Company"] = Finalmergeddata["Delivery Zone_x"]
Finalmergeddata["Expected Charge as per ShopX (Rs.)"] = Finalmergeddata["Our Amount5"]
Finalmergeddata["Charges Billed by Courier Company (Rs.)"] = Finalmergeddata["Total Amount (Rs.)"]
Finalmergeddata["Difference Between Expected Charges and Billed Charges (Rs.)"] = Finalmergeddata["Net"]

In [26]:
Finalmergeddata[Finalmergeddata["Order ID"]==2001808739]

,AWB Code (Airway Bill Number),Order ID,Chargeable Weight,Store House Pincode_x,Customer Area Code,Delivery Zone_x,Freight Type,Total Amount (Rs.),Total Weight,Store House Pincode_y,...,AWB Number,Total weight as per ShopX (KG),Weight slab as per ShopX (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per ShopX,Delivery Zone charged by Courier Company,Expected Charge as per ShopX (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
138,1091117437293,2001808739,1.63,121003,342012,d,Forward charges,179.8,0.22,121003,...,1091117437293,0.22,0.5,1.63,2.0,b,d,33.0,179.8,-146.8


In [27]:
Final = Finalmergeddata[["Order ID","AWB Number","Total weight as per ShopX (KG)","Weight slab as per ShopX (KG)",
    "Total weight as per Courier Company (KG)","Weight slab charged by Courier Company (KG)",
    "Delivery Zone as per ShopX","Delivery Zone charged by Courier Company",
    "Expected Charge as per ShopX (Rs.)","Charges Billed by Courier Company (Rs.)",
    "Difference Between Expected Charges and Billed Charges (Rs.)"]]

In [28]:
Final

,Order ID,AWB Number,Total weight as per ShopX (KG),Weight slab as per ShopX (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per ShopX,Delivery Zone charged by Courier Company,Expected Charge as per ShopX (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,135.0,135.0,0.0
1,2001806273,1091117222194,0.615,1.0,1.00,1.0,d,d,90.2,90.2,-0.0
2,2001806408,1091117222931,2.265,2.5,2.50,2.5,d,d,224.6,224.6,0.0
3,2001806458,1091117223244,0.700,1.0,1.00,1.0,b,b,61.3,61.3,0.0
4,2001807012,1091117229345,0.240,0.5,0.15,0.5,d,d,45.4,45.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...
167,2001812941,1091118551656,0.500,0.5,0.73,1.0,b,d,33.0,90.2,-57.2
168,2001809383,1091117614452,0.607,1.0,0.50,0.5,b,d,48.8,86.7,-37.9
169,2001820978,1091120922803,0.515,1.0,0.50,0.5,b,d,61.3,45.4,15.9
170,2001811475,1091121844806,0.689,1.0,0.50,0.5,e,b,112.1,33.0,79.1


In [29]:
Final2=pd.DataFrame(Finalmergeddata.groupby("Status")["Net"].count())
Final2.head()

,Net
Status,
Correctly Charged,21
Over Charged,92
Under Charged,11


In [30]:
Final2["Amount"] = Finalmergeddata.groupby("Status")["Net"].sum()
Final2

,Net,Amount
Status,,
Correctly Charged,21,0.0
Over Charged,92,-5680.7
Under Charged,11,481.5


In [31]:
Final2 = Final2.reset_index()

In [32]:
Final2

,Status,Net,Amount
0,Correctly Charged,21,0.0
1,Over Charged,92,-5680.7
2,Under Charged,11,481.5


In [33]:
with pd.ExcelWriter("Logistics_FINAL.xlsx") as writer:
    Final.to_excel(writer,sheet_name = "Summary")
    Final2.to_excel(writer,sheet_name = "Calculations")

In [34]:
pd.read_excel("Logistics_FINAL.xlsx")

,Unnamed: 0,Order ID,AWB Number,Total weight as per ShopX (KG),Weight slab as per ShopX (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per ShopX,Delivery Zone charged by Courier Company,Expected Charge as per ShopX (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,0,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,135.0,135.0,0.0
1,1,2001806273,1091117222194,0.615,1.0,1.00,1.0,d,d,90.2,90.2,0.0
2,2,2001806408,1091117222931,2.265,2.5,2.50,2.5,d,d,224.6,224.6,0.0
3,3,2001806458,1091117223244,0.700,1.0,1.00,1.0,b,b,61.3,61.3,0.0
4,4,2001807012,1091117229345,0.240,0.5,0.15,0.5,d,d,45.4,45.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
119,167,2001812941,1091118551656,0.500,0.5,0.73,1.0,b,d,33.0,90.2,-57.2
120,168,2001809383,1091117614452,0.607,1.0,0.50,0.5,b,d,48.8,86.7,-37.9
121,169,2001820978,1091120922803,0.515,1.0,0.50,0.5,b,d,61.3,45.4,15.9
122,170,2001811475,1091121844806,0.689,1.0,0.50,0.5,e,b,112.1,33.0,79.1
